# get_freq_mutations test Ovarian

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import CPTAC.Ovarian as ov

Welcome to the CPTAC data service package. Available datasets may be
viewed using CPTAC.list_data(). In order to access a specific data
set, import a CPTAC subfolder using either 'import CPTAC.Dataset' or
'from CPTAC import Dataset'.
******
Version: 0.3.1
******
Loading Ovarian CPTAC data:
Loading clinical data...
Loading cnv data...
Loading phosphoproteomics data...
Loading proteomics data...
Loading somatic_38 data...
Loading transcriptomics data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until June 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter embargo() to open the webpage for more details.


In [22]:
def get_frequently_mutated(somatic_df, omics_mutations_df, cutoff=.1):  
    unique_genes = somatic_df['Gene'].unique() # get series of all mutated genes
    freq_mutated = []
    gene_and_freq_d = {}
    
    # get total tumor patients
    tumors = omics_mutations_df.loc[omics_mutations_df['Sample_Status'] == 'Tumor']
    total_tumor_patients = len(tumors)
    
    #find sample (col or index) in gene_mutated (samples represent patients)
    #gene_mutated: Endometrial and Colon samples found in column 1, Ovarian found in index
    gene = 'PTEN'
    gene_mutated = somatic_df.loc[somatic_df['Gene'] == gene]
    ovarian = False
    sample = 0
    if gene_mutated.columns[0] == 'Gene':
        sample = gene_mutated.index
        ovarian = True
        
    if ovarian == True:
        print('ovarian')
        for gene in unique_genes:
            gene_mutated = somatic_df.loc[somatic_df['Gene'] == gene]
            num_gene_mutated = len(sample.unique())
            if (num_gene_mutated / total_tumor_patients) > cutoff:
                freq_mutated.append(gene)
    
    #find frequently mutated
    else:
        for gene in unique_genes:
            gene_mutated = somatic_df.loc[somatic_df['Gene'] == gene]
            num_gene_mutated = len(gene_mutated[gene_mutated.columns[sample]].unique())
            if (num_gene_mutated / total_tumor_patients) > cutoff:
                freq_mutated.append(gene)
    
    """
    # make dictionary and sort into descending order based on num mutations
    for gene in freq_mutated:
        gene_mutated = somatic.loc[somatic['Gene'] == gene]
        num_gene_mutated = len(gene_mutated['Patient_Id'].unique())     
        gene_and_freq_d[gene] = num_gene_mutated    
    """    
                   
    return freq_mutated

In [17]:
#get df
somatic = ov.get_mutations()
clinical = ov.get_clinical()
prot = ov.get_proteomics()

gene = 'TP53'
omics_mutations = ov.append_mutations_to_omics(mutation_genes=gene, omics_df=prot, omics_genes=gene)
gene_mutated = somatic.loc[somatic['Gene'] == 'PTEN']

omics_mutations.head()

,TP53_proteomics,TP53_Mutation,TP53_Location,Sample_Status
Sample_ID,,,,
S002,-1.600876,[Frame_Shift_Del],[p.D61*],Tumor
S006,-1.685874,[Frame_Shift_Del],[p.S90Vfs*55],Tumor
S007,0.846640,[Missense_Mutation],[p.R273C],Tumor
S009,0.419432,[Missense_Mutation],[p.I195T],Tumor
S011,-2.459607,[Frame_Shift_Ins],[p.C124Lfs*25],Tumor


In [30]:
#get column of sample = patient
gene = 'PTEN'
gene_mutated = somatic.loc[somatic['Gene'] == gene]
col_of_sample = 0
if gene_mutated.columns[0] == 'Gene':
    col_of_sample = gene_mutated.index.unique()
len(col_of_sample)

4

In [14]:
#Check num tumor patients
tumors = omics_mutations.loc[omics_mutations['Sample_Status'] == 'Tumor']
total_tumor_patients = len(tumors)
print('total_tumor_patients: ',total_tumor_patients)

total_tumor_patients:  83


Test 1: cutoff = .3

In [27]:
list1 = get_frequently_mutated(somatic, omics_mutations)

ovarian


In [28]:
print(len(list1))
list1

0


[]

In [8]:
#How many genes were frequently mutated compared to all mutated genes
print('freq_mutated: ',len(list1))
total_genes = somatic['Gene'].unique()
print('total_genes: ',len(total_genes))

freq_mutated:  15
total_genes:  10499


In [9]:
list1

['TTN',
 'TP53',
 'MUC16',
 'NF1',
 'MUC4',
 'MUC17',
 'CTAGE1',
 'LPA',
 'MUC12',
 'SPATA31D1',
 'AHNAK',
 'RGPD3',
 'RANBP2',
 'ARHGAP5',
 'LRRC37A3']

In [18]:
# check genes in freq_mutated list
gene = 'LRRC37A3'
total_tumor_patients = 97
cutoff = .1
gene_mutated = somatic.loc[somatic['Gene'] == gene]
num_gene_mutated = len(gene_mutated)
print(num_gene_mutated, ' / ', total_tumor_patients, ' ', (num_gene_mutated / total_tumor_patients))
if (num_gene_mutated / total_tumor_patients) > cutoff:
    print('true')
else:
    print('false')

26  /  97   0.26804123711340205
true
